In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request
import codecs

In [3]:
#Code from stackoverflow on reading html tables
response = urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_film_sequels_by_box-office_improvement')
html = response.read()
soup = BeautifulSoup(html)

/Users/Spirapple/anaconda/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [54]:
#soup.find("table")
#table.find_all("tr")

In [99]:
table = soup.find("table", attrs = {"class":"sortable wikitable"})

In [100]:
headings = [th.get_text() for th in table.find("tr").find_all("th")]

In [101]:
headings

['Title',
 'Year',
 'Gross (North America)',
 'Title (original)',
 'Year (original)',
 "Original's gross (North America)",
 'Improvement']

In [56]:
#This yields a list of zip objects I don't know how to work with
datasets=[]

for row in table.find_all("tr")[2:]:
    dataset = zip(headings, (td.get_text() for td in row.find_all("td")))
    datasets.append(dataset)

In [94]:
#While this yields a list of zip objects I don't know how to work with
datasets=[]

for row in table.find_all("tr")[1:]:
    dataset = [td.get_text() for td in row.find_all("td")]
    #Note: parens make generator object, [] get you the results in a list instead
    datasets.append(dataset)

In [95]:
#hell=(next(gen) for gen in datasets)
datasets[1]

['Desperado',
 '1995',
 '$25,405,445[3]',
 'El Mariachi',
 '1992',
 '$2,040,920[3]',
 '1,170%']

In [102]:
sequelDF=pd.DataFrame(data=datasets, columns = headings)

In [103]:
sequelDF.head()

,Title,Year,Gross (North America),Title (original),Year (original),Original's gross (North America),Improvement
0,The Boondock Saints II: All Saints Day,2009,"$10,273,188[1]",The Boondock Saints,1999,"$30,471[2]","33,714%"
1,Desperado,1995,"$25,405,445[3]",El Mariachi,1992,"$2,040,920[3]","1,170%"
2,Clerks II,2006,"$24,148,068[4]",Clerks,1994,"$3,151,130[5]",677%
3,The Color of Money,1986,"$52,293,982[6]",The Hustler,1961,"$7,600,000[7]",588%
4,Terminator 2: Judgment Day,1991,"$204,843,345[8]",The Terminator,1984,"$38,371,200[8]",434%


In [139]:
#sequelDF.shape() #I don't know why this isn't working, dammit.
len(sequelDF) #70 rows, probably 7 columns?
#P.S.this does just cover the first table, as intended.

70

In [105]:
import re

In [155]:
#Function to remove the brackets and punctuation in the Gross metrics

#So there's easy ways to take out only the numbers (ex: re, tkinter),
#but I need to remove the brackets first.

#x="$219,614,612[81]"

def numerify(x):
    x2 = re.sub("[\(\[].*?[\)\]]", "", x)#stackoverflow
    x3= re.findall("[0-9]+",x2)
    return("".join(x3))



In [156]:
testing = sequelDF['Gross (North America)'].map(lambda x:numerify(x))
#This works as intended!

In [158]:
sequelDF['Original\'s gross (North America)']= sequelDF['Original\'s gross (North America)'].map(lambda x:numerify(x))
sequelDF['Gross (North America)']= sequelDF['Gross (North America)'].map(lambda x:numerify(x))

In [160]:
sequelDF.to_csv("data/Sequel.csv", sep="|", header=True, index=False)
#sequelDF.head()

In [162]:
newseq = pd.read_csv("data/Sequel.csv", sep="|")

In [163]:
newseq.head() #Excellent, everything went well.

,Title,Year,Gross (North America),Title (original),Year (original),Original's gross (North America),Improvement
0,The Boondock Saints II: All Saints Day,2009,10273188,The Boondock Saints,1999,30471,"33,714%"
1,Desperado,1995,25405445,El Mariachi,1992,2040920,"1,170%"
2,Clerks II,2006,24148068,Clerks,1994,3151130,677%
3,The Color of Money,1986,52293982,The Hustler,1961,7600000,588%
4,Terminator 2: Judgment Day,1991,204843345,The Terminator,1984,38371200,434%


In [119]:
int("$219,614,612") #Doesn't work, good to know

ValueError: invalid literal for int() with base 10: '219,614,612'

In [117]:
?int()

In [53]:
#print("{}".format(datasets[1]))
next(datasets[1])#gets each as you iterate through; awkward?

StopIteration: 

In [38]:
for d in datasets:
    print(d)

TypeError: 'generator' object is not subscriptable

In [98]:
headings

['Title',
 'Year',
 'Gross (North America)',
 'Title (original)',
 'Year (original)',
 "Original's gross (North America)",
 'Improvement',
 ['The Boondock Saints II: All Saints Day',
  '2009',
  '$10,273,188[1]',
  'The Boondock Saints',
  '1999',
  '$30,471[2]',
  '33,714%'],
 ['Desperado',
  '1995',
  '$25,405,445[3]',
  'El Mariachi',
  '1992',
  '$2,040,920[3]',
  '1,170%'],
 ['Clerks II',
  '2006',
  '$24,148,068[4]',
  'Clerks',
  '1994',
  '$3,151,130[5]',
  '677%'],
 ['The Color of Money',
  '1986',
  '$52,293,982[6]',
  'The Hustler',
  '1961',
  '$7,600,000[7]',
  '588%'],
 ['Terminator 2: Judgment Day',
  '1991',
  '$204,843,345[8]',
  'The Terminator',
  '1984',
  '$38,371,200[8]',
  '434%'],
 ['Tron: Legacy',
  '2010',
  '$172,062,763[9]',
  'Tron',
  '1982',
  '$33,000,000[10]',
  '421%'],
 ['Austin Powers: The Spy Who Shagged Me',
  '1999',
  '$206,040,086[11]',
  'Austin Powers: International Man of Mystery',
  '1997',
  '$53,883,989[11]',
  '282%'],
 ['Rambo: First Blo

In [29]:
?zip()

In [96]:
?pd.DataFrame